In [ ]:
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
from backtesting.test import SMA, GOOG
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np

In [ ]:
df_fesx = pd.read_csv("./data/fesx_M15_2020-06-20_F.US.DSXM20.scid_BarData.txt", sep=", ", engine = "python")
df_fesx["Datetime"] = pd.to_datetime(df_fesx.Date + "-" + df_fesx.Time,format="%Y/%m/%d-%H:%M:%S")
df_fesx = df_fesx[['Datetime','Open', 'High', 'Low', 'Last', 'Volume']].rename(columns={"Last":"Close"})#.set_index("Datetime")
df_fesx = df_fesx.rename(columns={"Last":"Close"})
df_fesx.head()

In [ ]:
rolling_n = 2
min_diff = 0


diff_high = df_fesx.High.diff().reset_index()
diff_low = df_fesx.Low.diff()
diff_low.index = diff_low.index -(rolling_n)
diff_low = diff_low.reset_index()
diff_df = pd.merge(diff_high, diff_low, on="index", how="left").rename(columns={"High":"diff_high", "Low":"diff_low"})


# For every n elements: high(idx - n)> ... >high(idx - 1)>high(idx - 0) => diff(idx) < 0
# n consecutive elements with decreasing high: rolling(n).max() < 0
diff_df['high_max'] = diff_df.diff_high.rolling(rolling_n).max()
diff_df['low_min'] = diff_df.diff_low.rolling(rolling_n).min()

diff_df['Open'] = False

diff_df.loc[(diff_df.high_max < min_diff) & (diff_df.low_min > min_diff), 'Open'] = True


fig = go.Figure(data= [
    go.Scatter(x=df_fesx.index, y=df_fesx.High,name = 'High',mode="lines+markers", marker_color = "green"),
    go.Scatter(x=df_fesx.index, y=df_fesx.Low,name = 'Low',mode="lines+markers", marker_color = "red"),
    go.Scatter(x=df_fesx[diff_df.Open].index, y=df_fesx[diff_df.Open].High,name = 'Open',mode="markers", marker_color = "orange", marker_size = 2 ),
    
    go.Candlestick(x=df_fesx.index,
                open=df_fesx['Open'],
                high=df_fesx['High'],
                low=df_fesx['Low'],
                close=df_fesx['Close'],
                name = 'Candlestick'
                )
    ]
    )

fig.update_layout(
    xaxis_rangeslider_visible= False
)



for idx, row in df_fesx[diff_df.Open].iterrows():
   exact_point = False
   high = row['High']
   #print(idx, high)    
   # Check if the next point after a open is the same value => Dont consider next point as a touch point
   if  high == df_fesx.loc[idx+1,'High']:
      continue
   # Check if the next point after a open is lower than the open
   elif high > df_fesx.loc[idx+1,'High']:
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High >= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   # Check if the next point after a open is higher than the open
   elif high < df_fesx.loc[idx+1,'High']:     
      df_touch_line =  df_fesx[(df_fesx.index > idx) & (df_fesx.High <= high)]
      if df_touch_line.shape[0] == 0:
         continue
      if df_touch_line.High.iloc[0] == high:
         exact_point = True
   
   if not exact_point:
      # Linear interpolation
      y = df_touch_line.High.iloc[0]
      x = df_touch_line.iloc[0].name
      xo = x -1
      yo = df_fesx.loc[xo,'High']
      m = (y - yo)/(x - xo)
      dy = high - yo
      dx = dy/m
      next_idx = xo + dx
   else:
      next_idx = df_touch_line.iloc[0].name


   fig.add_trace(
      go.Scatter(x=[idx, next_idx], y=[high, high],name = 'level',mode="lines", marker_color = "red", showlegend=False)
   )
          

fig.show()

# Check 1060